# Development Notebook For MIMIC-III -> SQLite backend

First, lets connect to SQL-database for MIMIC-III found under `/Project_MVP/_datasets/mimiciii_sqlite`.

In [1]:
import sqlite3
sqlite_file = "/project/projectdirs/m1532/Projects_MVP/_datasets/mimiciii_sqlite/mimic3.db"

In [2]:
conn = sqlite3.connect(sqlite_file)
c = conn.cursor()

The following cell contains all SQL commands used in this notebook:

In [9]:
# SQL COMMANDS
get_tables = "SELECT name FROM sqlite_master WHERE type='table';"
get_noteevents_vars = "SELECT sql FROM sqlite_master WHERE name='noteevents';"
get_notes_forpatient = "SELECT hadm_id, chartdate, category, text FROM noteevents WHERE subject_id='{pid}' ORDER BY chartdate DESC;"

Lets check the names of tables in the database.

In [4]:
# Get Table Names
c.execute(get_tables)
rows = c.fetchall()
print(rows)

[('admissions',), ('callout',), ('caregivers',), ('chartevents',), ('cptevents',), ('d_cpt',), ('d_icd_diagnoses',), ('d_icd_procedures',), ('d_items',), ('d_labitems',), ('datetimeevents',), ('diagnoses_icd',), ('drgcodes',), ('icustays',), ('inputevents_cv',), ('inputevents_mv',), ('labevents',), ('microbiologyevents',), ('noteevents',), ('outputevents',), ('patients',), ('prescriptions',), ('procedureevents_mv',), ('procedures_icd',), ('services',), ('transfers',), ('admission_suicide',), ('time',), ('new',), ('all2',), ('new_1',), ('new_2',)]


In [5]:
import json

# Make JSON From Rows
def rows_to_json(rows, c_labels, path):
    json_data = {}
    for i,r in enumerate(rows):
        json_data[i] = {}
        for j,c in enumerate(c_labels):
            json_data[i][c] = str(r[j])
    with open(path, 'w') as f: json.dump(json_data, f)

### CASE: Notes APP
The notes web app being develop will display all medical notes for a given patient. The final output structure for the data fetched
from the database should be a JSON file with the following fields:

```
PID.json
ID: HADM, CHARTDATE, CATEGORY, TEXT
# SORTED BY CHARTDAT IN DESCENDING ORDER
```

Lets check the names of columns in the noteevents table.

In [6]:
# Get Note-Events Column names
c.execute(get_noteevents_vars)
rows = c.fetchall()
print(rows)

[('CREATE TABLE noteevents\n(\n    row_id integer NOT NULL,\n    subject_id integer NOT NULL,\n    hadm_id integer,\n    chartdate timestamp(0) ,\n    charttime timestamp(0) ,\n    storetime timestamp(0) ,\n    category varchar(50) ,\n    description varchar(255) ,\n    cgid integer,\n    iserror character(1) ,\n    text text ,\n    PRIMARY KEY (row_id)\n)',)]


In [13]:
# Get Note-Events For All HADMs of a given Patient.
c.execute(get_notes_forpatient.format(pid="85"))
rows = c.fetchall()
#print(rows)

In [12]:
# Save Sample Json
json_path = "/global/project/projectdirs/m1532/www/MVP-NEWT/backend/{pid}.json"
rows_to_json(rows, "hadm_id, chartdate, category, text".split(', '), json_path.format(pid="85"))

In [14]:
c.close()